<p align="center">
	<img src="https://user-images.githubusercontent.com/63207451/114284722-45901b80-9a52-11eb-8a0c-e99fc8681436.gif" height="90">
<p/>

# **IRGUI ILYAS, NGAKAM TCHEUMBE PESCIANY LAFORTUNE**

# Détection du COVID-19 dans les images radiographiques avec CNN & Tuning des hyperparamètres 

## Importation des bibliothèques

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
from imutils import paths
import os
import keras

from keras.layers import Conv2D, MaxPool2D, Dropout, Flatten, Dense
from keras.models import Sequential
from keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from keras.wrappers.scikit_learn import KerasClassifier

from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split, GridSearchCV


## Chargement du dataset

### Le dataset utilisé contient 25 images Normales et 25 images Covid en noir et blanc. 
### Nous avons importé le dataset dans le drive pour l'utiliser plus tard.

In [ ]:
imagePaths = list(paths.list_images("/content/drive/MyDrive/Colab Notebooks/dataset"))
imgs = []
labels = []


for imagePath in imagePaths:

	label = imagePath.split(os.path.sep)[-2]
	
	image = cv2.imread(imagePath)
	image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # Transformation en RGB(rouge vert bleu)
	image = cv2.resize(image, (224, 224)) # 224x224 pixels
	
	imgs.append(image)
	labels.append(label)
 
imgs = np.array(imgs) / 255.0 # La Normalisation 
labels = np.array(labels)

In [ ]:
# Encodage des labels en binaire
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels) # Convertion des classes vectorielles en classes binaires

# Partition du dataset en training and testing splits avec 80% training et 20% pour le testing
(trainX, testX, trainY, testY) = train_test_split(imgs, labels,
	test_size=0.20, stratify=labels, random_state=42)

## Création du modèle 

### Dans notre cas, nous avons fait le tuning de tous les hyperparamètres (sauf le learning rate) en même temps.
### Pour cela, nous avons donc définit les valeurs par défaut de certains hyperparamètres qui doivent être utilisés par d'autres.

In [ ]:
def create_model(activation = 'relu', neurons=32, optimizer='adam', dropout_rate=0.0, weight_constraint=0):
   model = Sequential()
   model.add(Conv2D(neurons, kernel_size=(3,3), activation = activation, input_shape=(224,224,3)))
   model.add(MaxPool2D(pool_size=(2, 2)))
   model.add(Flatten())
   model.add(Dropout(dropout_rate))
   model.add(Dense(neurons*2, activation= activation))
   model.add(Dense(2,activation='sigmoid'))
   model.compile(loss="binary_crossentropy", optimizer = optimizer, metrics=["accuracy"])
   return model

## Après plusieurs tests avec de nombreuses valeurs pour nos hyperparamètres, nous avons repéré la valeur idéale et avons ajusté nos listes à deux valeurs pour chacun des hyperparamètres pour gagner en temps.

In [ ]:
model = KerasClassifier(build_fn=create_model, batch_size=5, epochs=10, verbose=0) #Create model
 
#Define grid search parameters
batch_size=[8,10]
epochs = [20,25]
activation = ['softmax','relu']
neurons = [25,32]
optimizer = ['Adam', 'Adadelta']
dropout_rate = [0.2,0.5]

## Ensuite nous avons définit notre dictionnaire et appliqué GridSearchCV

In [ ]:
param_grid = dict(batch_size=batch_size, 
                  epochs=epochs, 
                  activation = activation, 
                  neurons = neurons,
                  optimizer = optimizer,
                  dropout_rate = dropout_rate)

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(trainX, trainY)

/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


## Summurize Results

In [ ]:
print("Best_parameters: %f,%s" %(grid_result.best_score_, grid_result.best_params_))

means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']

for mean ,stdev, param in zip(means, stds, params):
  print("%f(%f) avec: %r" %(mean,stdev,param))

Best_parameters: 1.000000,{'activation': 'relu', 'batch_size': 8, 'dropout_rate': 0.2, 'epochs': 25, 'neurons': 32, 'optimizer': 'Adam'}
0.371795(0.090655) avec: {'activation': 'softmax', 'batch_size': 8, 'dropout_rate': 0.2, 'epochs': 20, 'neurons': 25, 'optimizer': 'Adam'}
0.371795(0.090655) avec: {'activation': 'softmax', 'batch_size': 8, 'dropout_rate': 0.2, 'epochs': 20, 'neurons': 25, 'optimizer': 'Adadelta'}
0.628205(0.090655) avec: {'activation': 'softmax', 'batch_size': 8, 'dropout_rate': 0.2, 'epochs': 20, 'neurons': 32, 'optimizer': 'Adam'}
0.500000(0.157019) avec: {'activation': 'softmax', 'batch_size': 8, 'dropout_rate': 0.2, 'epochs': 20, 'neurons': 32, 'optimizer': 'Adadelta'}
0.371795(0.090655) avec: {'activation': 'softmax', 'batch_size': 8, 'dropout_rate': 0.2, 'epochs': 25, 'neurons': 25, 'optimizer': 'Adam'}
0.500000(0.157019) avec: {'activation': 'softmax', 'batch_size': 8, 'dropout_rate': 0.2, 'epochs': 25, 'neurons': 25, 'optimizer': 'Adadelta'}
0.371795(0.090655

## Le score optimal obtenu est de : 100% avec les hyperparamètres suivants :
## **{'activation': 'relu', 'batch_size': 8, 'dropout_rate': 0.2, 'epochs': 25, 'neurons': 32, 'optimizer': 'Adam'}**